In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [8]:
path = os.path.dirname(os.getcwd()) + "\\data\\"

In [10]:
data = pd.read_csv(path + "telco.csv")

In [24]:
desc = pd.read_excel(path + "field_descriptions.xlsx")

In [27]:
desc["Description"][3]

'Milliseconds offset of start time for the xDR (first frame timestamp)'

In [22]:
for i in data.columns:
    val = data[i].shape[0] - data[i].nunique()
    print(f"{i:15s} ==> {val}")

Bearer Id       ==> 15293
Start           ==> 140004
Start ms        ==> 149001
End             ==> 143598
End ms          ==> 149001
Dur. (ms)       ==> 60476
IMSI            ==> 42736
MSISDN/Number   ==> 43145
IMEI            ==> 42731
Last Location Name ==> 104454
Avg RTT DL (ms) ==> 147871
Avg RTT UL (ms) ==> 149279
Avg Bearer TP DL (kbps) ==> 108248
Avg Bearer TP UL (kbps) ==> 135473
TCP DL Retrans. Vol (Bytes) ==> 95602
TCP UL Retrans. Vol (Bytes) ==> 114881
DL TP < 50 Kbps (%) ==> 149901
50 Kbps < DL TP < 250 Kbps (%) ==> 149917
250 Kbps < DL TP < 1 Mbps (%) ==> 149928
DL TP > 1 Mbps (%) ==> 149916
UL TP < 10 Kbps (%) ==> 149903
10 Kbps < UL TP < 50 Kbps (%) ==> 149916
50 Kbps < UL TP < 300 Kbps (%) ==> 149933
UL TP > 300 Kbps (%) ==> 149938
HTTP DL (Bytes) ==> 86474
HTTP UL (Bytes) ==> 90059
Activity Duration DL (ms) ==> 47441
Activity Duration UL (ms) ==> 43709
Dur. (ms).1     ==> 27130
Handset Manufacturer ==> 149831
Handset Type    ==> 148605
Nb of sec with 125000B < Vol DL 

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     